We need to download nltk (even if it comes with Acaconda) and tell python what the path is, otherwise we can't use it.

In [134]:
nltk.download("wordnet", "./nltk_data/")
nltk.data.path.append('./nltk_data/')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to ./nltk_data/...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Now we'll clean the data by removing any punctuation, converting the ingredients to lower case using Lemmatization, removing stop words and removing non-ascii characters.
After doing this however, I'm not sure how to convert ingredients_string back to a type that the code below can use.

In [135]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer
from string import printable
st = set(printable)

df = pd.read_json("./train.json")

df['ingredients_string'] = [', '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in df['ingredients']]
#df['ingredients_string'] = df['ingredients_string'].str.replace('[{}]'.format('!"#$%;&\'()*+-./:<=>?@[\\]^_`{|}~'), ',')
#df['ingredients_string'] = df['ingredients_string'].str.lower()
pat = r'\b(?:{})\b'.format('|'.join(stop))
df['ingredients_string'] = df['ingredients_string'].str.replace(pat, '')
df['ingredients_string'] = df['ingredients_string'].str.replace(r'\s+', ' ')
df["ingredients_string"] = df["ingredients_string"].apply(lambda x: ''.join([" " if  i not in  st else i for i in x]))

df.head()

,cuisine,id,ingredients,ingredients_string
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...","romaine lettuce, black olives, grape tomatoes,..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...","plain flour, ground pepper, salt, tomato, grou..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...","egg, pepper, salt, mayonaise, cooking oil, gre..."
3,indian,22213,"[water, vegetable oil, wheat, salt]","water, vegetable oil, wheat, salt"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...","black pepper, shallot, cornflour, cayenne pepp..."


In [136]:
from sklearn.cross_validation import train_test_split

X = df['ingredients_string'].tolist()
y = df['cuisine'].tolist()

#new_X = []
#for l in X:
#    new_X.append(', '.join(l))
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=20)

In [137]:
from sklearn.feature_extraction.text import *

vectorizer = TfidfVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)  

tfidf_transformer = TfidfTransformer() 

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [138]:
from sklearn.preprocessing import LabelEncoder
e = LabelEncoder()
y = e.fit_transform(df.cuisine)

In [139]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(X_train_tfidf, y_train) 

X_test_counts = vectorizer.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
y_pred = logistic.predict(X_test_tfidf)

In [140]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
acc

0.774732872407291